In [262]:
import pandas as pd
import glob
import numpy as np
import re
import csv
import random
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import statsmodels.api as sm
import statsmodels.formula.api as fsm
from sklearn.ensemble import RandomForestRegressor
from sklearn import (cross_validation, cluster, datasets, decomposition, ensemble, preprocessing, metrics)

#Import Cleaned Data

In [263]:
input_data = pd.read_csv('../Restaurants/cleaned_res_data2.csv').drop(['Unnamed: 0'],axis=1)
input_data.head()

,PRODUCT,FARM,CT,PRICE,DATE,file_name,YEAR,MONTH,PRODUCT_LIST,QUANTITY,UNIT,PRICE_PER_UNIT,CAT
0,super baby chicory mix,marin,3 lb,24,01.05.15.,../LISTS by GROUP/Bay Area Resturants/2015/Cit...,15,1,"[super, baby, chicory, mix]",3,lb,8,chicory
1,super baby chicory mix,marin,3 lb,24,01.09.15.,../LISTS by GROUP/Bay Area Resturants/2015/Cit...,15,1,"[super, baby, chicory, mix]",3,lb,8,chicory
2,super baby chicory mix,marin,3 lb,24,01.13.15.,../LISTS by GROUP/Bay Area Resturants/2015/Cit...,15,1,"[super, baby, chicory, mix]",3,lb,8,chicory
3,super baby chicory mix,marin,3 lb,24,01.16.15.,../LISTS by GROUP/Bay Area Resturants/2015/Cit...,15,1,"[super, baby, chicory, mix]",3,lb,8,chicory
4,super baby chicory mix,marin,3 lb,24,01.20.15.,../LISTS by GROUP/Bay Area Resturants/2015/Cit...,15,1,"[super, baby, chicory, mix]",3,lb,8,chicory


In [264]:
input_data[['YEAR','MONTH']] = input_data[['YEAR','MONTH']].astype(float)

In [265]:
#calculate continuous date variable
input_data['DATEFAC'] = (input_data['YEAR']-12) + (input_data['MONTH']-1)/12

In [266]:
#drop unnecessary fields
data = input_data.drop(['DATE', 'CT', 'file_name', 'PRODUCT_LIST','MONTH','YEAR'], axis=1)[input_data['PRICE_PER_UNIT'].notnull()].reset_index(drop=True)
data['intercept']=1.0

In [267]:
#get dummies
#data_dummy = pd.get_dummies(data, columns=[u'FARM', u'UNIT', U'CAT'])
data_dummy = pd.get_dummies(data, columns=[u'UNIT','CAT'])
data_dummy = pd.concat([data_dummy, data['CAT']],axis=1)

In [268]:
# data_dummy.apply(lambda x['CAT']: 'kale' if 'a else x)
# data_dummy['CAT'] = data_dummy['CAT'].map(lambda x: 'chard' if 'chard' in x else x)
# data_dummy['CAT'] = data_dummy['CAT'].map(lambda x: 'broccoli' if 'broccoli' in x else x)
data_dummy.head().T

,0,1,2,3,4
PRODUCT,super baby chicory mix,super baby chicory mix,super baby chicory mix,super baby chicory mix,super baby chicory mix
FARM,marin,marin,marin,marin,marin
PRICE,24,24,24,24,24
QUANTITY,3,3,3,3,3
PRICE_PER_UNIT,8,8,8,8,8
DATEFAC,3,3,3,3,3
intercept,1,1,1,1,1
UNIT_bu,0,0,0,0,0
UNIT_bulbs,0,0,0,0,0
UNIT_ct,0,0,0,0,0


In [269]:
#  Add in fields for Quantity * Unit
data_dummy[u'QUANTITY_UNIT_bu'] = data_dummy[[u'QUANTITY','UNIT_bu']].product(axis=1)
data_dummy[u'QUANTITY_UNIT_bulbs'] = data_dummy[[u'QUANTITY','UNIT_bulbs']].product(axis=1)
data_dummy[u'QUANTITY_UNIT_ct'] = data_dummy[[u'QUANTITY','UNIT_ct']].product(axis=1)
data_dummy[u'QUANTITY_UNIT_flat'] = data_dummy[[u'QUANTITY','UNIT_flat']].product(axis=1)
data_dummy[u'QUANTITY_UNIT_lb'] = data_dummy[[u'QUANTITY','UNIT_lb']].product(axis=1)
data_dummy[u'QUANTITY_UNIT_leaves'] = data_dummy[[u'QUANTITY','UNIT_leaves']].product(axis=1)

In [270]:
ols_data = data_dummy.copy()
y = ols_data.pop('PRICE_PER_UNIT')
mdl_data = ols_data.drop(['PRODUCT','CAT_bean','CAT','FARM','UNIT_bu','QUANTITY_UNIT_bu', 'PRICE'],axis=1)
mdl = sm.OLS(y, mdl_data)
results = mdl.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         PRICE_PER_UNIT   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     1229.
Date:                Tue, 18 Aug 2015   Prob (F-statistic):               0.00
Time:                        13:59:30   Log-Likelihood:                -58436.
No. Observations:               19821   AIC:                         1.169e+05
Df Residuals:                   19795   BIC:                         1.171e+05
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
QUANTITY                 1.8696      0.098     19.111      0.000         1.678     2.061
DATEFAC                 -0.0120      0.060     -0.200      0.842        -0.130     0.106
intercept              -26.5317      1.666    -15.926      0.000       -29.797   -23.266
UNIT_bulbs               0.0042      0.016      0.265      0.791        -0.027     0.035
UNIT_ct                 21.8376      1.178     18.533      0.000        19.528    24.147
UNIT_flat               26.3923      0.584     45.199      0.000        25.248    27.537
UNIT_lb                 32.4527      1.166     27.825      0.000        30.167    34.739
UNIT_leaves             -0.0814      0.004    -18.211      0.000        -0.090    -0.073
CAT_bouquet              7.8413      2.027      3.869      0.000         3.869    11.814
CAT_broccoli             6.3406      1.220      5.196      0.000         3.949     8.733
CAT_chard                4.4407      1.206      3.683      0.000         2.077     6.804
CAT_chicory              1.3732      1.199      1.145      0.252        -0.977     3.723
CAT_cucumber            -0.2481      1.257     -0.197      0.844        -2.712     2.216
CAT_fine green           2.1451      1.195      1.794      0.073        -0.198     4.488
CAT_fruit                2.4203      1.234      1.961      0.050         0.001     4.840
CAT_herb                17.2835      1.200     14.408      0.000        14.932    19.635
CAT_kale                 3.1044      1.200      2.586      0.010         0.752     5.457
CAT_melon                5.2371      1.362      3.846      0.000         2.568     7.906
CAT_pepper               0.3713      1.283      0.289      0.772        -2.144     2.886
CAT_potato               5.8700      1.203      4.878      0.000         3.511     8.229
CAT_root                 4.0538      1.196      3.388      0.001         1.709     6.399
CAT_salad mix            1.5430      1.198      1.288      0.198        -0.805     3.891
CAT_spinach              1.7306      1.211      1.429      0.153        -0.643     4.104
CAT_squash               4.1297      1.342      3.077      0.002         1.499     6.760
QUANTITY_UNIT_bulbs      0.0507      0.191      0.265      0.791        -0.324     0.426
QUANTITY_UNIT_ct        -1.8054      0.098    -18.418      0.000        -1.998    -1.613
QUANTITY_UNIT_flat      26.3923      0.584     45.199      0.000        25.248    27.537
QUANTITY_UNIT_lb        -2.3514      0.098    -24.014      0.000        -2.543    -2.160
QUANTITY_UNIT_leaves    -1.2211      0.067    -18.211      0.000        -1.353    -1.090
==============================================================================
Omnibus:                    15201.723   Durbin-Watson:                   0.279
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           836668.25

#Randomized PCA

In [271]:
# try CountVectorizer: best adj r2 for 30 components using RandomizedPCA, set maxfeatures to 50 ~ .722
vec_machine = CountVectorizer(max_features=30)
word_matrix = vec_machine.fit_transform(data_dummy['PRODUCT'])
words = pd.DataFrame(word_matrix.todense().astype(np.float))
words.columns = vec_machine.get_feature_names()
print data_dummy.shape
print words.shape

(19821, 37)
(19821, 30)


In [272]:
# try TFIDF: best adj r2 for 30 components using RandomizedPCA, set maxfeatures to 50 ~ .73, .735 for maxf = 60 and comp=40
#Random Forest: r2 ~.96 for max_features = 15, n_components = 5
vec_machine = TfidfVectorizer(max_features=15)
word_matrix = vec_machine.fit_transform(data_dummy['PRODUCT'])
words = pd.DataFrame(word_matrix.todense().astype(np.float))
words.columns = vec_machine.get_feature_names()
print data_dummy.shape
print words.shape

(19821, 37)
(19821, 15)


In [273]:
rpca = decomposition.RandomizedPCA(n_components=5)
rpca.fit(words)

RandomizedPCA(copy=True, iterated_power=3, n_components=5, random_state=None,
       whiten=False)

In [274]:
from sklearn.preprocessing import scale

multi_regress = words.copy()

X_pca = rpca.fit_transform(scale(multi_regress.as_matrix()))

In [289]:
ols_data = rf_data.copy()
y = ols_data['PRICE_PER_UNIT']
mdl_data = ols_data.drop(['PRICE_PER_UNIT','PRODUCT','CAT_bean','CAT','FARM','UNIT_bu','QUANTITY_UNIT_bu', 'PRICE'],axis=1)
mdl = sm.OLS(y, mdl_data)
results = mdl.fit()
results.summary()
ols_data['PREDICTIONS'] = results.predict()

In [292]:
ols_data['ABS_ERROR'] = np.round(np.abs(ols_data['PRICE_PER_UNIT'] - ols_data['PREDICTIONS']),decimals=2)
ols_data['TOTAL_PRICE_PRED'] = np.round(ols_data['QUANTITY']*ols_data['PREDICTIONS'],decimals=2)
ols_data['TOTAL_PRICE_DIFF'] = np.round(np.abs(ols_data['PRICE'] - ols_data['TOTAL_PRICE_PRED']),decimals=2)
ols_data['PERC_ERROR'] = np.round(ols_data['ABS_ERROR']/ols_data['PRICE_PER_UNIT'],decimals=2)
np.mean(ols_data['PERC_ERROR'])

1.1220543867615158

# K-Fold Cross Validation

In [285]:
features = pd.DataFrame(X_pca)

rf_data = pd.concat([data_dummy, features], axis=1).drop(['DATEFAC','UNIT_bulbs','QUANTITY_UNIT_bulbs'],axis=1).reset_index(drop=True)
#rf_data = pd.concat([data_dummy, features], axis=1).drop(['DATEFAC','UNIT_bulbs','QUANTITY_UNIT_bu'],axis=1).reset_index(drop=True)
#rf_data = data_dummy.drop(['DATEFAC','FARM_coachella','UNIT_bulbs','CAT_bean','QUANTITY_UNIT_bu'],axis=1).reset_index(drop=True)

In [276]:
products = rf_data['PRODUCT'].unique()
len(products)

640

In [283]:
kf = cross_validation.KFold(len(products), n_folds=5, shuffle=True)
results = []
dollar_errors = []
perc_errors = []
med_error = []
data_pred = rf_data.copy()
data_pred['PREDICTIONS'] = -1
for train_index, test_index in kf:
    train_products = products[train_index]
    train_data = rf_data[rf_data['PRODUCT'].isin(train_products)]
    test_products = products[test_index]
    #print test_products
    test_data = rf_data[rf_data['PRODUCT'].isin(test_products)]
    x_train = train_data.drop(['PRICE_PER_UNIT','PRODUCT', 'PRICE','CAT_bean','FARM','CAT'],axis=1)
    y_train = train_data['PRICE_PER_UNIT']
    x_test = test_data.drop(['PRICE_PER_UNIT','PRODUCT', 'PRICE','CAT_bean','FARM','CAT'],axis=1)
    y_test = test_data['PRICE_PER_UNIT']
    regr = LinearRegression()
    #regr = RandomForestRegressor(n_estimators=5)
    regr.fit(x_train, y_train)
    predictions = regr.predict(x_test)
#     for (prod, pred, true) in zip(test_products, predictions, y_test):
#         if pred != true:
#             print "product:", prod
#             print "prediction:", pred
#             print "true y:", true
#             print "-------------------------------------------"
#             dollar_errors.append(true - pred)
#             perc_errors.append((true - pred)/true)
    data_pred.iloc[test_data.index,-1] = np.round(predictions,decimals=2)
    results.append(regr.score(x_test, y_test))
print "average score:", np.mean(results)

average score: -1.6601410401e+20


In [284]:
results

[-8.3007052005181437e+20,
 0.69494571837141583,
 -157548074.8219544,
 0.21561771657824669,
 0.47402606668110958]

In [278]:
data_pred['ABS_ERROR'] = np.round(np.abs(data_pred['PRICE_PER_UNIT'] - data_pred['PREDICTIONS']),decimals=2)
data_pred['TOTAL_PRICE_PRED'] = np.round(data_pred['QUANTITY']*data_pred['PREDICTIONS'],decimals=2)
data_pred['TOTAL_PRICE_DIFF'] = np.round(np.abs(data_pred['PRICE'] - data_pred['TOTAL_PRICE_PRED']),decimals=2)
data_pred['PERC_ERROR'] = np.round(data_pred['ABS_ERROR']/data_pred['PRICE_PER_UNIT'],decimals=2)
np.mean(data_pred['TOTAL_PRICE_DIFF'])

2.3251990313304072

In [279]:
np.mean(data_pred.PERC_ERROR)

0.13214671308208467